# 협업 알고리즘

- 가장 기본적인 CF 알고리즘은 이웃(neighbor)을 전체 사용자로 한다. 즉 현재 사용자와 취향이 비슷한 사용자 그굽을 따로 선정하지 않고 모든 사용자의 평점을 가지고 예측한다. 구체적으로는 아래와 같이 작동한다 

    1. 모든 사용자 간의 평가의 유사도를 계산(상관계수, 코사인 유사도) 등을 사용 
    2. 현재 추천 대상이 된느 사람과 다른 사용자의 유사도를 추출
    3. 현재 사용자가 평가하지 않은 모든 아이템에 대해서 현재 사용자의 예상 평가값을 구한다. 예상 평가값은 다른 사용자의 해당 아이템에 대한 평가(평점)를 현재 사용자와 그 사용자와의 유사도로 가중해서 평균을 낸다 
    4. 아이템 중에서 예상 평가값이 가장 높은 N개의 아이템을 추천 

In [1]:
import numpy as np
import pandas as pd

In [2]:
# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('./data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

### 기본 CF 알고리즘

In [4]:
# train set의 모든 가능한 사용자 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity 

matrix_dummy = rating_matrix.copy().fillna(0) # 코사인 유사도를 게산하기 위해서 rating 값을 matrix_dummy 에 복사,NaN값을 0으로 바꿔준다 
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy) # 모든 사용자 간의 코사인 유사도를 구한다 . 
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index) # index 저장 

In [5]:
# 주어진 영화의 (movie_id) 가중평균 rating을 계산하는 함수, 
# 가중치는 주어진 사용자와 다른 사용자 간의 유사도(user_similarity)
def CF_simple(user_id, movie_id):
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        movie_ratings = rating_matrix[movie_id].copy()
        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        # 사람들에 대해서는 가중 평균 계산에서 빼기 위함이다
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings = movie_ratings.dropna()
        # 현재 영화를 평가하지 않은 사용자의 similarity값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
        # 현재 영화를 평가한 모든 사용자의 가중평균값 구하기
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

In [6]:
# 정확도 계산
score(CF_simple)

1.017589485450691

-  단순 CF 알고리즘으로 에측치를 게산한 경우, 성병이나 직업별 평균을 사용한 경우보다 상딩히 게산된다. 즉, 평가 경향이 비슷한(유사도가 높은) 사용자에게 가중치를 더 주어서 평균을 계산하면 더 정확한 예측을 할 수 있음을 알 수 있다. 